### импортируем библиотеки

In [1]:
from pprint import pprint
import json
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

### функция разбиения строки с зарплатой на мин, макс и валюту

In [2]:
def apd_salary(i):
    
    mi, ma = '', ''
    result = ['','','']
    
    i = i.replace(u'\xa0', u'')
    i = i.replace(u'.', u'')
    
    if re.search('от ', i):
        min = re.findall('\d+', i)
        mi = min[0]
        
    if re.search('до ', i):
        max = re.findall('\d+', i)
        ma = max[0]
 
    if re.search('\d+-\d+', i):
        min = re.findall('\d+', i)
        max = re.findall('-\d+', i)
        max[0] = max[0].replace(u'-', u'')
        mi = min[0]
        ma = max[0]
        
    result[0] = mi
    result[1] = ma
    result[2] = i[-3:]
    
    return(result) 

### ПАРСИМ hh.ru

In [13]:
def parsing(html, site):
    res_block = html.find('div',{'class':'vacancy-serp vacancy-serp_xs-mode'})
    res_list = res_block.findChildren(recursive=False)

    results=[]
    for vak in res_list:
        data = {}
        vak_salary = ['','','']
        try:    
            vak_name = vak.find('a',{'class':'bloko-link HH-LinkModifier'}).text#.findChild()
            vak_link = vak.find('a',{'class':'bloko-link HH-LinkModifier'})['href']
        except:
            continue           
    
        try:
            vak_s = vak.find('div',{'class':'vacancy-serp-item__compensation'}).getText()
            vak_salary = apd_salary(vak_s)
        except:
            pass

        data['name'] = vak_name
        data['salary_min'] = vak_salary[0]
        data['salary_max'] = vak_salary[1]
        data['salary_valuta'] = vak_salary[2]
        data['link'] = vak_link
        data['site'] = site

        results.append(data)    
        
    return(results)

main_link = 'https://hh.ru'
#user_input = input()
link = main_link + '/search/vacancy?L_save_area=true&clusters=true&enable_snippets=true&search_field=name&text=курьер&showClusters=false'#+user_input
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
# &page=8
response = requests.get(link, headers=headers).text
html = bs(response,'lxml')

#pprint(response)
#html

res_block = html.find('a',{'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})

# СЧИТАЕМ СКРОЛЛИНГ
scr = html.findAll('a',{'class':'bloko-button HH-Pager-Control'})
scr = int(scr[-1].text)

i = 1
results=[]
while res_block:
    res_block = html.find('a',{'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})
    print(int(100*i/(scr+1)), ' %')
    results += parsing(html, main_link)
    link1 = link + '&page=' + str(i)
    i += 1
    response = requests.get(link1, headers=headers).text
    html = bs(response,'lxml')

# pprint(results)

with open('data.json', 'w') as outfile:
    json.dump(results, outfile)

print('DONE!')

2  %
4  %
7  %
9  %
12  %
14  %
17  %
19  %
21  %
24  %
26  %
29  %
31  %
34  %
36  %
39  %
41  %
43  %
46  %
48  %
51  %
53  %
56  %
58  %
60  %
63  %
65  %
68  %
70  %
73  %
75  %
78  %
80  %
82  %
85  %
87  %
90  %
92  %
95  %
97  %
DONE!


### парсим superjob.ru

In [16]:
def parsingSJ(html, site):
    res_block = html.find('div',{'class':'_3zucV'})
    res_list = res_block.findChildren(recursive=False)

    results=[]
    for vak in res_list:
        data = {}
        vak_salary = ['','','']
        try:    
            vak_name = vak.find('a',{'class':'bloko-link HH-LinkModifier'}).text#.findChild()
            vak_link = vak.find('a',{'class':'bloko-link HH-LinkModifier'})['href']
        except:
            continue           
    
        try:
            vak_s = vak.find('div',{'class':'vacancy-serp-item__compensation'}).getText()
            vak_salary = apd_salary(vak_s)
        except:
            pass

        data['name'] = vak_name
        data['salary_min'] = vak_salary[0]
        data['salary_max'] = vak_salary[1]
        data['salary_valuta'] = vak_salary[2]
        data['link'] = vak_link
        data['site'] = site

        results.append(data)    
        
    return(results)


main_link = 'https://www.superjob.ru'
#user_input = input()
link = main_link + '/vakansii/povar.html?geo%5Bt%5D%5B0%5D=4'#+user_input
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
# &page=8
response = requests.get(link, headers=headers).text
html = bs(response,'lxml')

#pprint(response)
#html

res_block = html.findAll('span',{'class':'_3IDf-'})#[-1].getText()
max_page = 0
for i in res_block:
    try:
        if int(i.text):
            if max_page < int(i.text):
                max_page = int(i.text)
    except:
        pass

i = 1
results=[]

while i<max_page+1:
    print(int(100*i/(max_page+1)), ' %')
    results += parsingSJ(html, main_link)
    i += 1
    link1 = link + '&page=' + str(i)
    response = requests.get(link1, headers=headers).text
    html = bs(response,'lxml')

with open('data.json', 'w') as outfile:
    json.dump(results, outfile)

print('DONE!')

6  %
12  %
18  %
25  %


KeyboardInterrupt: 

### смотрим, что сохранилось в файле с результатами

In [12]:
with open('data.json') as fh: 
    data = json.load(fh)
len(data)
#data

2000

In [48]:
df1 = pd.DataFrame(data)
df1.head(25)

,name,salary_min,salary_max,salary_valuta,link,site
0,Data Scientist,200000,,руб,https://pushkino.hh.ru/vacancy/35011897?query=...,https://hh.ru
1,Аналитик (команда Поиска),,,,https://pushkino.hh.ru/vacancy/35121709?query=...,https://hh.ru
2,"Стажёр (Анализ данных, data science)",,,,https://pushkino.hh.ru/vacancy/35431519?query=...,https://hh.ru
3,Junior Quantitative Researcher,250000,250000,руб,https://pushkino.hh.ru/vacancy/31844648?query=...,https://hh.ru
4,Python Data Scientist,150000,,руб,https://pushkino.hh.ru/vacancy/35296762?query=...,https://hh.ru
5,IT руководитель команды (направление Data Scie...,,,,https://pushkino.hh.ru/vacancy/33975505?query=...,https://hh.ru
6,Junior ML Engineer / Data scientist,80000,120000,руб,https://pushkino.hh.ru/vacancy/35459810?query=...,https://hh.ru
7,System Analyst,4000,,USD,https://pushkino.hh.ru/vacancy/34961834?query=...,https://hh.ru
8,"Специалист / data scientist (big data, прогнос...",100000,,руб,https://pushkino.hh.ru/vacancy/35360203?query=...,https://hh.ru
9,Junior Data Scientist,,,,https://pushkino.hh.ru/vacancy/35625371?query=...,https://hh.ru


In [17]:
main_link = 'https://www.superjob.ru'
link = main_link + '/vacancy/search/?keywords=data%20science&geo%5Bt%5D%5B0%5D=4'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

response = requests.get(link, headers=headers).text
html = bs(response,'lxml')

res_block = html.find('div',{'class':'iJCa5 _1JhPh _2gFpt _1znz6 _1LlO2 _2nteL'})
res_list = res_block.findChildren(recursive=False)
    
#pprint(res_list)

In [21]:
results=[]

for vak in res_list:
    vak_name = vak.find('div',{'class':'f-test-vacancy-item'})

    data = {}
    vak_salary = ['','','']
    try:    
        vak_name = vak_name.find('a',{'class':'bloko-link HH-LinkModifier'}).text#.findChild()
       # vak_link = vak.find('a',{'class':'bloko-link HH-LinkModifier'})['href']
    except:
        continue           
    
    pprint(vak_name)

[<div class="rI-XC"><div class="iJCa5 _58AhZ _2p0TS _2nteL"><div class="_3zucV undefined _2tR2A _1s0tF"><div class="GPKTZ _1o0Xp _1tH7S"><span class="_3mfro _1ZlLP _2JVkc _2VHxz"><span>Найдено 7 вакансий</span></span><div class="_1o0Xp _1tH7S"><a class="icMQ_ _1_Cht f-test-button-Na_karte f-test-link-Na_karte" href="/vacancy/search/?keywords=data%20science&amp;geo%5Bt%5D%5B0%5D=4&amp;onMap=true" target="_self"><span class="qTHqo _2h9me DYJ1Y _2FQ5q zB43c _2kFbd" tabindex="-1"><span class="_23m0W"><span class="_3f5Nw"><svg class="_2pknO _3ZJ4b" viewbox="0 0 24 24"><use xlink:href="#outline_map"></use></svg></span><span class="_3IDf-">На карте</span></span></span></a><div class="iJCa5 _3Tuzw _1xn13 _58AhZ X7voU _2nteL"><div class="f-test-vacancy-subscripion"><button class="_1_Cht f-test-button-Sohranit_poisk" tabindex="0" type="button"><span class="qTHqo _2h9me _3eVdb _2rOnu" tabindex="-1"><span class="_23m0W"><span class="_3IDf-">Сохранить поиск</span></span></span></button></div></div>

In [104]:
    data = {}    
    vak_salary = ['','','']
    try:    
        vak_name = vak.find('a',{'class':'_2g1F-'}).text#.findChild()
        #vak_link = vak.find('a',{'class':'bloko-link HH-LinkModifier'})['href']
    except:
        continue           
    
    try:
        vak_s = vak.find('div',{'class':'vacancy-serp-item__compensation'}).getText()
        vak_salary = apd_salary(vak_s)
    except:
        pass

    data['name'] = vak_name
    data['salary_min'] = vak_salary[0]
    data['salary_max'] = vak_salary[1]
    data['salary_valuta'] = vak_salary[2]
    data['link'] = vak_link
    data['site'] = site

    results.append(data)    
        
print(results)

[]
